In [15]:
import pandas as pd

c1 = ['1-1', '1-2', '1-3','1-4', '1-5', '1-6']
c2 = ['2015-05-20 09:00:00', '2015-06-20 10:42:00', '2015-07-20 11:42:00', '2015-08-20 09:00:00', '2015-08-23 10:42:00', '2015-09-01 11:42:00']
c3 = ['joe', 'bob', 'joe', 'joe', 'bob', 'joe']
c4 = ['joe', 'bob', 'joe', 'joe', 'bob', 'joe']
c5 = [30.0, 25.0, 40.0, 23.0, 40.0, 20.0]
c2 = map(lambda x: pd.to_datetime(x), c2)

fields = ['id', 'timestamp_start', 'person1', 'person2', 'minutes']

s = {
    fields[0]: pd.Series(c1),
    fields[1]: pd.Series(c2),
    fields[2]: pd.Series(c3),
    fields[3]: pd.Series(c4),
    fields[4]: pd.Series(c5),
}
df_test = pd.DataFrame(s)

from bokeh.charts import Scatter, show, output_notebook 
from bokeh.palettes import RdPu9
from bokeh.models import HoverTool

hover = HoverTool(
    tooltips = [
    ("ID", "@id"),
    ("Date", "@timestamp_start"),
    ("Person 1", "@person1"),
    ("Person 2", "@person2"),
    ('Duration', '@minutes'),
])
tools = [hover] 

fields = ['SEQUENCE_NO_', 'timestamp_start', 'CHILD_STARTED_USER', 'CARE_COMPLETED_USER', 'timestamp_diff_min']
# p = TimeSeries(df_test, x='timestamp_start', y='minutes', tools=tools, builder_type='point')
p = Scatter(df_test, x='timestamp_start', y='minutes', tools=tools,)
# p.line(df_test, x='timestamp_start', y='minutes', )
output_notebook()
show(p)

Loading BokehJS ...